In [1]:
import psycopg2 as ps
import pandas as pd
import Preprocessing
import time
import numpy as np
from collections import Counter
import fuzzywuzzy as fuz

## Getting first time data

In [ ]:
## Connect to database

conn = ps.connect(
    database='traffictwt', user='postgres', password='root')
cursor = conn.cursor()

q = '''SELECT * FROM TWEETS
        WHERE TWEET_CREATED >= '2018-01-01' 
        AND TWEET_CREATED < '2021-01-01'; '''
cursor.execute(q)
results = cursor.fetchall()
colnames = [desc[0] 
        for desc in cursor.description]
cursor.close()
conn.close()

df = pd.DataFrame(results, columns=colnames)

In [4]:
start = time.time()
df_clean = Preprocessing.clean_text(df, 'tweet_text')
end = time.time()
print(str(end-start), 'ms')

11.453316688537598 ms


In [5]:
df_clean.clean_text = df_clean.clean.copy()
df_clean = df_clean.drop('clean', axis=1)

In [6]:
# shift column 'Name' to first position
clean_text = df_clean.pop('clean_text') 
# insert column using insert(position,column_name,
# first_column) function
df_clean.insert(6, 'clean_text', clean_text)

## Using data csv

In [99]:
short_df = df_clean[['tweet_id', 'clean_text', 
    'isincident', 'isaccident', 'isobstacle', 'isdanger', 'incident_desc']].copy()

In [212]:
short_df = pd.read_csv('tweets-2018-2020-clean.csv')
short_df.drop('Unnamed: 0', axis=1, inplace=True)
## Fill NaN with None (if condition is false, then None)
short_df = short_df.where(pd.notnull(short_df), None)

In [3]:
short_df

,tweet_id,clean_text,isincident,isaccident,isobstacle,isdanger,incident_desc,incident_desc1
0,1013816364220518402,precaucion calle 50 anegada via,None,None,None,None,None,None
1,951132392906870784,recorrido mañanitas,None,None,None,None,None,None
2,1010612149826342914,inundada calle 70 san fransisco,True,None,None,True,inundacion,None
3,1013816934142562304,inundada calle 74 san francisco precaucion ruta,True,None,None,True,inundacion,None
4,1022561878969671680,inundada avenida frente mercado marisco,True,None,None,True,inundacion,None
...,...,...,...,...,...,...,...,...
87101,1344759940305612820,5km h desplazan autos via transistmica altura ...,True,None,True,False,None,reporte velocidad
87102,1344773829290287105,recibe resumen noticias capsulas video canal y...,False,False,False,False,None,None
87103,1344780610594856967,celebremos viven casa forma virtual expresemos...,None,None,None,None,None,None
87104,1344810167276363781,registra accidente transito via centenario alt...,True,True,False,False,accidente,None


In [4]:
len(short_df)

87106

In [598]:
## Count values still not analyzed
len(short_df[short_df.isincident.isnull()])

14060

In [6]:
pd.set_option('display.max_columns', 500)
pd.set_option("max_colwidth", 200)
pd.set_option('display.max_rows', 500)
pd.set_option('display.min_rows', 100)

In [591]:
len(short_df[extract])

122

In [593]:
short_df.loc[extract, 'isincident'] = False
short_df.loc[extract, 'isaccident'] = False
short_df.loc[extract, 'isobstacle'] = False
short_df.loc[extract, 'isdanger'] = False
# short_df.loc[extract, 'incident_desc1'] = 'area acordonada'

In [594]:
## Check the true with the keyword
extract = short_df.clean_text.str.contains('redes sociales') & short_df.isincident.isnull()
short_df[extract]

,tweet_id,clean_text,isincident,isaccident,isobstacle,isdanger,incident_desc,incident_desc1


In [428]:
counts = Counter(' '.join(short_df[short_df.isincident.isnull()].clean_text).split())
# short_df.isaccident.isnull() & short_df.clean_text.notnull()
freq = pd.DataFrame(counts.most_common(80), columns=['word', 'count'])
freq

,word,count
0,via,2993
1,panama,2712
2,altura,1411
3,nacional,1114
4,auto,996
5,tu,941
6,personas,898
7,san,895
8,seguridad,888
9,policia,847


# NTLK

In [429]:
from nltk import everygrams, word_tokenize, FreqDist

In [595]:
df = short_df[short_df.isincident.isnull()]
df['combinations'] = df.clean_text.apply(lambda x: [' '.join(word) for word in everygrams(word_tokenize(x), 2, 2)])

<ipython-input-595-a6a91ea9925a>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['combinations'] = df.clean_text.apply(lambda x: [' '.join(word) for word in everygrams(word_tokenize(x), 2, 2)])


In [596]:
df1 = pd.DataFrame(list(FreqDist([''.join(y) for x in df['combinations'] for y in x]).items()), columns=['combination', 'freq'])

In [597]:
df1.sort_values('freq', ascending=False).head(50)

,combination,freq
72,panama oeste,427
71,direccion panama,311
2286,via interamericana,263
3016,san miguelito,229
866,panama centro,192
2287,interamericana altura,188
310,cinta costera,185
1216,provincia panama,179
532,precaucion ruta,171
1007,puente americas,169


# Accidentes

In [276]:
len(short_df[extract])

230

In [38]:
counts = Counter(' '.join(short_df[extract].clean_text).split())
# short_df.isaccident.isnull() & short_df.clean_text.notnull()
freq = pd.DataFrame(counts.most_common(20), columns=['word', 'count'])
freq.head(5)

,word,count


In [420]:
short_df.loc[extract, 'isincident'] = True
short_df.loc[extract, 'isaccident'] = True
# short_df.loc[extract, 'isobstacle'] = False
short_df.loc[extract, 'isdanger'] = True
short_df.loc[extract, 'incident_desc'] = 'incidente'

In [418]:
len(short_df[extract])

50

In [421]:
extract = short_df.clean_text.str.contains('reportan.*') & short_df.isincident.isnull()
short_df[extract]

,tweet_id,clean_text,isincident,isaccident,isobstacle,isdanger,incident_desc,incident_desc1
201,948995781570256897,fuerte viento reportan puntos costa momentos precaucion area,None,None,None,None,None,None
202,948997472684306435,reportan fuertes vientos puntos costa,None,None,None,None,None,None
1974,958745221659054081,reportan movimiento sismico puntos panama reporte sector sentir,None,None,None,None,None,None
2819,962549153984376832,reportan fuerte movimiento sismico puntos panama envia tu reporte pudiste sentir estes,None,None,None,None,None,None
3197,965240139051040769,reportan movimiento sismico puntos panama envia tu reporte te encuentres pudiste sentir,None,None,None,None,None,None
3254,965421257125761024,reportan ganado soltura milla 21 carretera conduce guabito bocas toro precaucion señores conductores,None,None,None,None,None,None
5202,974286304086290434,seguimos reportando calle bambus obra suspendida mes alcalde indefinidamente concejo mun trabajando,None,None,None,None,None,None
5430,975489869865766914,exministro guillermo ferrufino capturan manejando efectos alcohol marco 65 prueba alcoholemia reportan licencia vencida,None,None,None,None,None,None
7976,986250541784236033,reportan usuarios bus ruta deja pasajeros tirados placa 8b 2913 nevera color rojo pavo amenaza usuarios,None,None,None,None,None,None
8768,990060654584324096,reportan largas filas terminal nacional transporte usuarios presentan problemas dirigirse panama oeste,None,None,None,None,None,None


In [336]:
## Specific rows
short_df.loc[753, 'isdanger'] = False

In [ ]:
## Save data into the same CSV (override)
short_df.to_csv('tweets-2018-2020-clean.csv', encoding='utf-8')

In [208]:
## Check the true with the keyword
extract = short_df.clean_text.str.contains('accidente[s]?') & short_df.isaccident==True
short_df[extract]

,tweet_id,clean_text,isincident,isaccident,isobstacle,isdanger,incident_desc,incident_desc1
14,947883288630300673,registra accidente transito autopista arraijan chorrera altura bique direccion interior pais,True,True,False,False,accidente,None
15,947885261849071616,imagenes accidente transito autopista arraijan chorrera altura bique direccion interior pais,True,True,False,False,accidente,None
17,947891590172901383,trafico afectado accidente transito autopista arraijan chorrera altura bique direccion interior pais,True,True,False,False,"tranque, accidente",None
18,947896257024004097,registra accidente transito autopista arraijan chorrera altura bique direccion interior via,True,True,False,False,accidente,None
19,947897042386505728,accidente transito autopista arraijan chorrera altura bique direccion interior pais via,True,True,False,False,accidente,None
20,947903020901257216,mantiene trafico afectado accidente transito autopista arraijan chorrera altura bique direccion interior pais,True,True,False,False,"tranque, accidente",None
21,947903052320858112,aguaceros city panama cuidado calles suficientes accidentes muertos cuidado 1 50pm,True,True,None,None,accidente,None
22,947904579659816960,heridos deja accidente transito carretera costa altura villa luisa provincia colon,True,True,False,False,accidente,None
24,947912385658064898,accidente transito autopista arraijan chorrera altura bique direccion interior pais via noticias,True,True,False,False,accidente,None
25,947912884964687872,accidente transito carretera costa altura villa luisa provincia colon deja 5 heridos,True,True,False,False,accidente,None


In [15]:
short_df.loc[extract, 'isincident'] = False
short_df.loc[extract, 'isaccident'] = False
# short_df.loc[extract, 'isobstacle'] = False
# short_df.loc[extract, 'isdanger'] = False
# short_df.loc[extract, 'incident_desc'] = None

# Inspect

In [372]:
## Inspect a certain keyword in CSV
short_df[extract].to_csv('colision-transito.csv', encoding='utf-8')

In [381]:
revision = pd.read_csv('colision-transito.csv')
revision = revision.where(pd.notnull(revision), None)

In [399]:
len(revision)

394

In [400]:
# short_df.loc[short_df.tweet_id.isin(revision[revision.isaccident==1].tweet_id), 'isaccident'] = True
# short_df.loc[short_df.tweet_id.isin(revision[revision.isaccident==0].tweet_id), 'isaccident'] = False
# short_df.loc[short_df.tweet_id.isin(revision[revision.isaccident==1].tweet_id), 'isincident'] = True
# short_df.loc[short_df.tweet_id.isin(revision[revision.isaccident==1].tweet_id), 'incident_desc'] = 'colision'
short_df.loc[short_df.tweet_id.isin(revision.tweet_id), :]

# Peligros

In [407]:
short_df.loc[extract, 'isincident'] = True
short_df.loc[extract, 'isaccident'] = False
short_df.loc[extract, 'isdanger'] = True
short_df.loc[extract, 'isobstacle'] = True
short_df.loc[extract, 'incident_desc'] = 'metro'

In [406]:
len(short_df[extract])

616

In [408]:
extract = short_df.clean_text.str.contains('metro ') & short_df.isincident.isnull()
short_df[extract]

,tweet_id,clean_text,isincident,isaccident,isobstacle,isdanger,incident_desc,incident_desc1


# Obstáculos

In [270]:
# len(short_df[short_df.isincident.isnull()])
len(short_df[extract])

67

In [546]:
counts = Counter(' '.join(short_df[extract].clean_text).split())
# short_df.isaccident.isnull() & short_df.clean_text.notnull()
freq = pd.DataFrame(counts.most_common(30), columns=['word', 'count'])
freq

,word,count
0,policia,270
1,nacional,175
2,via,61
3,policial,49
4,panama,44
5,policias,42
6,san,40
7,personas,32
8,area,25
9,altura,24


In [553]:
short_df.loc[extract, 'isincident'] = False
short_df.loc[extract, 'isaccident'] = False
short_df.loc[extract, 'isobstacle'] = False
short_df.loc[extract, 'isdanger'] = False
short_df.loc[extract, 'incident_desc1'] = ''

In [548]:
len(short_df[extract])

365

In [571]:
extract = short_df.clean_text.str.contains('panama oeste') & short_df.isincident.isnull()
short_df[extract]

,tweet_id,clean_text,isincident,isaccident,isobstacle,isdanger,incident_desc,incident_desc1
114,948308001710100482,auto avenida martires altura asamblea nacional direccion panama oeste precaucion,None,None,None,None,None,None
424,950161020714184704,retweeted full time vista panama oeste,None,None,None,None,None,None
1206,954389193223913472,trabajadores autoridad maritima panama oeste paro indefinido indican contralor firmar aguinaldo,None,None,None,None,None,None
1471,955929567641985025,matan conductor taxi supuestamente robarle cerro castillo chorrillo provincia panama oeste autoridades via,None,None,None,None,None,None
1570,956348169067073536,grupo taxistas cansados mop arregle calles decidieron tapar hueco principales calles burunga provincia panama oeste opinas,None,None,None,None,None,None
4780,972561465227010048,incidente barriada emperador provincia panama oeste via,None,None,None,None,None,None
5915,977359718602690561,pasando via puente americas panama oeste mueve,None,None,None,None,None,None
5918,977366226107322368,retweeted carla espitia pasando via puente americas panama oeste mueve,None,None,None,None,None,None
7056,982739369579679744,5 km h desplazan autos cinta costera 3 altura maracana direccion panama oeste,None,None,None,None,None,None
7646,984994047382220800,via panama oeste centenario carros hombros,None,None,None,None,None,None


# Save data

In [599]:
## Save data into the same CSV (override)
short_df.to_csv('tweets-2018-2020-clean.csv', encoding='utf-8')

In [9]:
# for i in range(2014, 2021):
#     df_clean[(df_clean.tweet_created>='%i-01-01'%i) & (df_clean.tweet_created<'%i-01-01'%(i+1))].to_csv('tweets_%i.csv'%(i), encoding='utf-8')
df_clean.to_csv('tweets-2018-2020.csv', encoding='utf-8')